In [1]:
# Load pickled data
import pickle
%run helpers.py

# TODO: fill this in based on where you saved the training and testing data
training_file = 'data/train.p'
testing_file = 'data/test.p'

with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
    
X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

import pandas as pd
signnames = pd.read_csv('signnames.csv')

### To start off let's do a basic data summary.
n_train = len(X_train)
n_test = len(X_test)
image_shape = X_train.shape[1]
n_classes = len(set(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 39209
Number of testing examples = 12630
Image data shape = 32
Number of classes = 43


In [2]:
# Training data
X_train_gray = np.empty( [n_train, image_shape, image_shape], dtype = np.int32)
for i, img in enumerate(X_train):
    X_train_gray[i,:,:] = cv2.equalizeHist(grayscale(img), (0, 254) )
# Test data
X_test_gray = np.empty( [n_test, image_shape, image_shape], dtype = np.int32)
for i, img in enumerate(X_test):
    X_test_gray[i,:,:] = cv2.equalizeHist(grayscale(img), (0, 254) )

In [3]:
from sklearn.model_selection import train_test_split

X_train_gray, X_valid_gray, y_train, y_valid = train_test_split(
    X_train_gray,
    y_train,
    test_size=0.10,
    random_state=1973)

In [4]:
from sklearn.preprocessing import LabelBinarizer

encoder = LabelBinarizer()
encoder.fit(y_train)
train_labels = encoder.transform(y_train)
valid_labels = encoder.transform(y_valid)
test_labels = encoder.transform(y_test)

# Change to float32, so it can be multiplied against the features in TensorFlow, which are float32
train_labels = train_labels.astype(np.float32)
valid_labels = valid_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [5]:
X_train_gray_flat = flatten_all_gray(X_train_gray)
X_valid_gray_flat = flatten_all_gray(X_valid_gray)
X_test_gray_flat = flatten_all_gray(X_test_gray)

X_train_gray_flat = normalize_grayscale(X_train_gray_flat)
X_valid_gray_flat = normalize_grayscale(X_valid_gray_flat)
X_test_gray_flat = normalize_grayscale(X_test_gray_flat)

In [8]:
import tensorflow as tf

tf.reset_default_graph()

TRAIN_DIR = 'final-cnn/'

with tf.Session() as sess:
    saver = tf.train.import_meta_graph(
        os.path.join(TRAIN_DIR, "checkpoint-20.meta"))
    saver.restore(
        sess, os.path.join(TRAIN_DIR, "checkpoint-20"))

    # These are the Minimal Ops we saved:
    output = tf.get_collection("output")[0]
    X = tf.get_collection("X")[0]
    y = tf.get_collection("y")[0]
    
    print(output)
    
    output_probability = tf.nn.softmax(output)
    predicted_class = tf.argmax(output_probability, dimension=1)
    is_correct_prediction = tf.equal(tf.argmax(output_probability, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct_prediction, tf.float32))

    valid_feed_dict = { X: X_test_gray_flat, y: test_labels}

    acc = sess.run(accuracy, feed_dict=valid_feed_dict)
    
    print(acc)

Tensor("cnn/add_3:0", shape=(?, 43), dtype=float32)
0.876722
